<a href="https://colab.research.google.com/github/fujie-cit/espnet-tts-finetune-example/blob/main/text_to_speech_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 特殊トークンを使った感情切り替え音声合成デモ

## 必要なパッケージのインストール

初回実行時は「RESTART RUNTIME」ボタンが表示されるので押してください．

In [ ]:
!pip install pyopenjtalk-prebuilt==0.3.0
!pip install parallel-wavegan 
!pip install git+https://github.com/fujie-cit/espnet.git@dev-fujie
!pip install espnet_model_zoo
!pip install -U numpy numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 11.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.0 MB/s eta 0:00:00
  Created wheel for parallel-wavegan: filename=parallel_wavegan-0.5.5-py3-none-any.whl size=72444 sha256=ecaf98f7ecbbd98b83719e8bd650dc86a09b77d689a2d3fd46c040808e48665f
  Stored in directory: /root/.cache/pip/wheels/7f/bd/95/08ea0ff486b17b4bcae062bdda3aeca710fd2f8d939e67afb5
Successfully built parallel-wavegan
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 13.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.1
    Uninstalling llvmlite-0.39.1:
      Successfully uninstalled llvmlite-0.39.1
  Attempting uninstall: numba
    Found existing installation: numba 0.56.4
    Uninstalling numba-0.56.4:
      Successfully uninstalled numba-0.56.4

## パッケージの読み込み

In [ ]:
from espnet2.bin.tts_inference import Text2Speech
import torch

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


## モデルの読み込み


In [ ]:
model_tag = "fujie/fujie_studies_tts_finetune_vits_raw_phn_jaconv_pyopenjtalk_prosody_with_special_token"
vocoder_tag = "parallel_wavegan/jsut_parallel_wavegan.v1"
device='cuda' if torch.cuda.is_available() else 'cpu'

text2speech = Text2Speech.from_pretrained(model_tag=model_tag, vocoder_tag=vocoder_tag, device=device)

Fetching 27 files:   0%|          | 0/27 [00:00<?, ?it/s]

Downloading...
From: https://drive.google.com/uc?id=1qok91A6wuubuz4be-P9R2zKhNmQXG0VQ
To: /root/.cache/parallel_wavegan/jsut_parallel_wavegan.v1.tar.gz
100%|██████████| 15.5M/15.5M [00:00<00:00, 58.0MB/s]
/usr/local/lib/python3.10/dist-packages/espnet2/gan_tts/vits/monotonic_align/__init__.py:19: UserWarning: Cython version is not available. Fallback to 'EXPERIMETAL' numba version. If you want to use the cython version, please build it as follows: `cd espnet2/gan_tts/vits/monotonic_align; python setup.py build_ext --inplace`
  warnings.warn(


## 音声合成例

In [ ]:
from IPython.display import display, Audio, Markdown

def generate(text, emotion):
  emo_dict = {'平静': '', '喜び': '<happy>', '怒り': '<angry>', '悲しみ': '<sad>'}
  tag = emo_dict.get(emotion, '')
  result = text2speech(f"{tag}{text}")
  display(Markdown(f"**{emotion}** {text}"))
  display(Audio(result["wav"].cpu(), rate=text2speech.fs))

In [ ]:
emotions = ["平静", "喜び", "怒り", "悲しみ"]
text = "あらゆる現実を全て自分の方へ捻じ曲げたのだ"
for emo in emotions:
  generate(text, emo)

Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"


dic.tar.gz: 0.00B [00:00, ?B/s]

Extracting tar file /usr/local/lib/python3.10/dist-packages/pyopenjtalk/dic.tar.gz


**平静** あらゆる現実を全て自分の方へ捻じ曲げたのだ

**喜び** あらゆる現実を全て自分の方へ捻じ曲げたのだ

**怒り** あらゆる現実を全て自分の方へ捻じ曲げたのだ

**悲しみ** あらゆる現実を全て自分の方へ捻じ曲げたのだ

In [ ]:
text = "えっ、それは良かったね！"
for emo in emotions:
  generate(text, emo)

**平静** えっ、それは良かったね！

**喜び** えっ、それは良かったね！

**怒り** えっ、それは良かったね！

**悲しみ** えっ、それは良かったね！

In [ ]:
text = "えっ、嘘でしょ？"
for emo in emotions:
  generate(text, emo)

**平静** えっ、嘘でしょ？

**喜び** えっ、嘘でしょ？

**怒り** えっ、嘘でしょ？

**悲しみ** えっ、嘘でしょ？

In [ ]:
text = "それは残念だったね"
for emo in emotions:
  generate(text, emo)

**平静** それは残念だったね

**喜び** それは残念だったね

**怒り** それは残念だったね

**悲しみ** それは残念だったね